<a href="https://colab.research.google.com/github/yeray142/AIML_class/blob/master/Week6/AIML_w6_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04


In [6]:
import pandas as pd
weather = pd.read_csv('data/weather_by_station.csv', index_col='date', parse_dates=True)
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"


In [7]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [8]:
weather.query('station == "GHCND:USW00094728"')\
    .pivot(columns='datatype', values='value')[['SNOW', 'PRCP']]\
    .sum()

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

In [9]:
dp = weather.query('datatype == "PRCP"').groupby(level=0).mean()

In [10]:
dp.head()

,value
date,
2018-01-01,0.00
2018-01-02,0.00
2018-01-03,0.00
2018-01-04,4.92
2018-01-05,12.30


In [11]:
dp.groupby(pd.Grouper(freq='Q')).sum()

,value
date,
2018-03-31,364.89
2018-06-30,334.89
2018-09-30,546.52
2018-12-31,473.97


In [12]:
dp.groupby(pd.Grouper(freq='Q')).count()

,value
date,
2018-03-31,90
2018-06-30,91
2018-09-30,92
2018-12-31,92


In [15]:
import numpy as np
dp.assign(
        total_prcp_in_q=lambda x: \
            x.groupby(pd.Grouper(freq='Q')).transform(np.sum),
        total_count_in=lambda x: \
            x.groupby(pd.Grouper(freq='Q')).value.transform('count'),
    )

,value,total_prcp_in_q,total_count_in
date,,,
2018-01-01,0.00,364.89,90
2018-01-02,0.00,364.89,90
2018-01-03,0.00,364.89,90
2018-01-04,4.92,364.89,90
2018-01-05,12.30,364.89,90
...,...,...,...
2018-12-27,0.00,473.97,92
2018-12-28,10.46,473.97,92
2018-12-29,18.66,473.97,92


In [16]:
fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903,low
2018-01-03,181.88,184.78,181.33,184.67,16886563,low
2018-01-04,184.90,186.21,184.10,184.33,13880896,low
2018-01-05,185.59,186.90,184.93,186.85,13574535,low
2018-01-08,187.20,188.90,186.33,188.28,17994726,low


In [27]:
fb_pt = fb.pivot_table(index='trading_volume', columns=fb.index.month, values='close', aggfunc='count')
fb_pt.columns.name = 'month'
fb_pt

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0
